# Lab 2: Sequence Alignment

### Name: Yuhang Chen (yuhangc3)


### Due March 18, 2020 11:59 PM

#### Preamble (Don't change this)

## Important Instructions - 

1. You are not allowed to use any built-in libraries for calculating Smith-Waterman alignment/score.
2. Please implement all the *graded functions* in main.py file. Do not change function names in main.py.
3. Please read the description of every graded function very carefully. The description clearly states what is the expectation of each graded function. 
4. After some graded functions, there is a cell which you can run and see if the expected output matches the output you are getting. 
5. The expected output provided is just a way for you to assess the correctness of your code. The code will be tested on several other cases as well.

In [1]:
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm 
import numpy as np
import pickle

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Sequence Alignment

In this lab, we will look into performing sequence alignment between genomic sequences.
As we discussed in class, this is a key computational task in genomics.
In particular, sequence alignment is used in the following two scenarios:
* When we sequence the DNA of an organism that we have never sequenced before, we need to align the reads to each other in order to recover the entire genome.
* When we sequence the DNA of an organism for which a reference genome is available (e.g., humans), we need to align the reads to the reference genome.

Abstractly, in the sequence alignment problem, we are given two sequences $x$ and $y$, and we want to place gaps (represented by ‘-’) in $x$ and $y$ so that the resulting sequences “line up well”.
For example, if $x = AGGCTAGTT$ and $y = AGCGAAGTTT$, a "good" alignment is 

```
AGGC-TA-GTT-
AG-CG-AAGTTT
```

As we discussed in class, the Smith-Waterman algorithm assigns scores/penalties to matches, mismatches, and gaps gaps, and then computes the alignment between the two sequences that maximizes the total score.

The Smith-Waterman algorithm performs *local* sequence alignment. This means that we are looking for a substring of x and a substring of y with the largest possible alignment score.
For example, if our scores are +1 for match, -1 for mismatch, -1 for gap and we want to align $x = CCCCGATTACAGGGG$ and $y = GGGGGATACACCCC$, then the best possible local alignment is

```
GATTACA
GAT_ACA
```

which has score 6-1=5. Notice that the gaps in the beginning and in the end don't 


### PacBio data

We will start with the same PacBio data from Lab 1. 
PacBio reads are typically long, and aligning them can be challenging in practice.

In [2]:
#reading PacBio data
with open('dna_reads_pac-bio.data', 'rb') as filehandle:
    dna_reads_pac=pickle.load(filehandle)

In [34]:
%run main.py

The following line creates an object from the class in *main.py*. **Do not change the class name and function headers!**

In [35]:
module = Lab2()

## Graded Function 1: smith_waterman_alignment  (10 marks)

Purpose - To perform local sequence alignment between two DNA sequences and identify sequence similarity using the Smith-Waterman algorithm. You should calculate alignment score between every two points in the sequences and record the maximum score.

Input - two sequences and a dictionary with penalties for match, mismatch and gap (e.g., `penalties={'match':1,'mismatch':-1,'gap':-1}`)

Output - an integer value which is the maximum smith waterman alignment score

In [6]:
penalties={'match':1,'mismatch':-1,'gap':-1}

In [7]:
# Note this may take some time to compute
print(module.smith_waterman_alignment(dna_reads_pac[0],dna_reads_pac[1],penalties))

--- 216.25250005722046 seconds ---
593


### Expected Output - 

593

As you noticed, finding the optimal alignment between two long PacBio reads takes a while. 
Imagine doing this for hundreds of thousands of reads!
Some of the indexing techniques that we will explore later in this lab can be used in practice to accelerate this process.

## Graded Function 2: print_smith_waterman_alignment  (10 marks)

Purpose - To perform local sequence alignment between two DNA sequences and return the resulting alignment in a nice fashion, like:

```
AGGC-TA-GTT-
AG-CG-AAGTTT
```

Input - two sequences and a dictionary with penalities for match, mismatch and gap

Output - return a tuple with two strings showing the two sequences with '-' representing the gaps

In [37]:
x = "MISPEL"
y = "MISSPELL"
module.print_smith_waterman_alignment(x,y,penalties)

('MI-SPEL', 'MISSPEL')

### Expected Output - 

``('MI-SPEL', 'MISSPEL')``

or 

``('MIS-PEL', 'MISSPEL')``

# Aligning reads to a (long) genome

While the Smith-Waterman algorithm can provide local alignments between two sequences of arbitrary lengths, it is too slow to be used to align reads to a long genome.
As we discussed in class, when we are trying to align reads to a long genome, we typically rely on an indexing scheme (based on hash functions, or a Python dictionary) to quickly identify matches.

We will consider two genome files.
The first one is a short fake genome in the file "fakegenome.fasta".

The second one is the *Saccharomyces cerevisiae* (Brewer's yeast) genome.
The *S. cerevisiae* genome was the first eukaryotic genome to be fully sequenced.
It contains 16 chromosomes for a total genome length of about 12 million base-pairs.

In [18]:
fakegenome_file=""
with open("fakegenome.fasta") as file:
    fakegenome_file=file.read()

saccha_file=""
with open("saccha.fasta") as file:
    saccha_file=file.read()

In [19]:
# let's print the fakegenome file and the beginning of the S. cerevisiae file:

print(fakegenome_file)

print()
print(saccha_file[:300])

>chr1
GATTACA
>chr2
CAGATTTACACATACA
>chr3
CACACACA


>chr1
CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACA
CATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTGGCCAACCTGTCTCTCAACTT
ACCCTCCATTACCCTGCCTCCACTCGTTACCCTGTCCCATTCAACCATACCACTCCGAAC
CACCATCCATCCCTCTACTTACTACCACTCACCCACCGTTACCCTCCAATTACCCATATC
CAACCCACTGCCACTTACCCTACCATTACCCTACCATCCACCATGACCTA


Notice that the chromosomes are separated by a line that only contains ">chrN", where N is the chromosome number

## Graded Function 3 : find_exact_matches(list_of_reads,genome_file)

Pupose - To check whether each of the reads in list_of_reads is present (exactly) somewhere in the genome and, if so, return the location. The location should be specified as "chr2:120000" (for a read that starts at position 120000 of chromosome 2)

Input - list of reads **of the same length** and a genome fasta file (converted into a single string)

Output - a list with the same length as list_of_reads, where the ith element is a list of all locations (starting positions) in the genome where the ith read appears. The starting positions should be specified using the "chr2:120000" format

Note: Avoid using Python packages and built-in functions to do search operations (such as the find function). The goal of this problem is for you to practice using Python dictionaries to build a genome index that can help finding matches quickly.

Note: Genomic locations should be spaced using 1-based indexing. For example, the first position of chromosome 3 should be specified as ``chr3:1`` (and not ``chr3:0``)

In [29]:
number = []
for i in range(0,10):
    number.append(str(i))
print(number)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [21]:
list_of_fake_reads = ['GATT','TACA','CACA']
print(module.find_exact_matches(list_of_fake_reads,fakegenome_file))

--- 0.0 seconds ---
[['chr1:1', 'chr2:3'], ['chr1:4', 'chr2:7', 'chr2:13'], ['chr2:9', 'chr3:1', 'chr3:3', 'chr3:5']]


### Expected Output - 

``[['chr1:1', 'chr2:3'], ['chr1:4', 'chr2:7', 'chr2:13'], ['chr2:9', 'chr3:1', 'chr3:3', 'chr3:5']]``

In [22]:
read0 = "CCACACCACACCCACACACCCACACACCACACCACACACCACACCACACC"
read1 = "CACACCACACCACACCCACACACACACATCCTAACACTACCCTAACACAG"
read2 = "CTCGCTGTCACTCCTTACCCGGCTTTCTGACCGAAATTAAAAAAAAAAAA"
read3 = "TTTAAACTTACGATTATGTGATTTGATGAGGTCAATCAACAGATTAACCA"
read4 = "CTGTATGGCTATACGATTATGTGGGCTACCAACAGATTGGTCACTTTCCT"
read5 = "GGGTCCGATGTTGGATTGAAATCCCAAGGTGCTATTTCTATATTTATATA"
list_of_reads = [read0,read1,read2,read3,read4]

print(module.find_exact_matches(list_of_reads,saccha_file))

--- 28.007336854934692 seconds ---
[['chr1:1'], ['chr1:35'], ['chr8:56', 'chr13:74'], ['chr2:753363']]


# Aligning reads with errors/mutations to a (long) genome

When the reads may have discrepancies with respect to the reference genome (which could be sequencing errors or mutations), we need to be more clever with our indexing.

In the following, we will use the same two genome files (fakegenome_file and saccha_file) from above, but consider reads with errors/mutations.

## Graded Function 4 : find_approximate_matches(list_of_reads,genome_file)

Purpose - To return the locations in the genome file which have the highest Smith-Waterman alignment score for each of the reads in list_of_reads. All reads in the list will have the same length, say $L$. For each read, your goal is to find a segment of length $L$ in the genome with the largest Smith-Waterman alignment score with the read. 

Notice that simply running Smith-Waterman between a read and every length-$L$ segment in the genome is impractical (and will take too long). Instead you should use an indexing scheme, based on Python dictionaries, to first identify candidate locations in the genome, and then use the Smith-Waterman algorithm to find the alignment score.

For Smith-Waterman, you should use penalties={'match':1,'mismatch':-1,'gap':-1}.

Input - list of reads of the same length and a genome fasta file (converted into a single string)

Output - a list with the same length as list_of_reads, where the ith element is a list of all locations (starting positions) in the genome which have the highest Smith-Waterman alignment score with the ith read in list_of_reads

Note: The location should be specified as "chr2:120000" (for the length-$L$ segment that starts at position 120000 of chromosome 2). As in Graded function 3, you should specify the position using 1-based indexing; i.e., the chromosome starts at position 1, not at 0).

Note: there can be more than one position for a particular read which have the same highest score. You should include all of them as a list.

**Hint:** For all cases that we will be checking, you can choose the substring length (for the genome index) to be $k = L/4$. We will only use lengths $L$ that are divisible by 4.

In [23]:
print(fakegenome_file)

>chr1
GATTACA
>chr2
CAGATTTACACATACA
>chr3
CACACACA



In [24]:
print(module.find_approximate_matches(["GATTACAT","CACAAACA"],fakegenome_file))

--- 0.0009963512420654297 seconds ---
--- 0.000997781753540039 seconds ---
--- 0.0 seconds ---
--- 0.0019941329956054688 seconds ---
--- 0.0 seconds ---
--- 0.0009975433349609375 seconds ---
--- 0.000997304916381836 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0009975433349609375 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0009963512420654297 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0009975433349609375 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---
--- 0.0009965896606445312 seconds ---
--- 0.0 seconds ---
--- 0.0 seconds ---


### Expected Output - 

``[['chr2:3'], ['chr2:9', 'chr3:1']]``

In [25]:
read0 = "TGCAGATTGCTCCTACGCTTGACAATGTCGGATCCGATACCGATCTGATTCATATCGATACAGTTAGTGCCATTAACGAGCAATTTCTAAGACTGCACTG"
read1 = "ACGTAAAAAATGTAGCAGACTCGATCTCCTCTTCTGATGAAATCCTAGTTCCTTCGAGACTCGCTGATGTTACGCTAGCATTCATGGAGGAGAATGACGC"
read2 = "AAGTGGAAAGAAAGAAGGGTGACAAGTTCGTCGCTTGTTTCACAAGATTACCAACGCCAGCCATATTGTAACATAGATGTATAACTAGAACAATTTACCA"
read3 = "CCACACCACACCCACACACCCACACACCACACCACACACCCACCACACCCACACACACACATCCTAACAACTACCCTAACACAGCCCTAATCTAACCCTG"

list_of_reads = [read0,read1,read2,read3]

print(module.find_approximate_matches(list_of_reads,saccha_file))

--- 0.05186057090759277 seconds ---
--- 0.04787135124206543 seconds ---
--- 0.04088902473449707 seconds ---
--- 0.03889584541320801 seconds ---
--- 0.05884408950805664 seconds ---
--- 0.08676719665527344 seconds ---
--- 0.06482553482055664 seconds ---
--- 0.0578455924987793 seconds ---
--- 0.0388946533203125 seconds ---
--- 0.06533145904541016 seconds ---
--- 0.037897586822509766 seconds ---
--- 0.03191399574279785 seconds ---
--- 0.02991795539855957 seconds ---
--- 0.04388236999511719 seconds ---
--- 0.03490614891052246 seconds ---
--- 0.030930042266845703 seconds ---
--- 0.046874284744262695 seconds ---
--- 0.03390932083129883 seconds ---
--- 0.03490781784057617 seconds ---
--- 0.03590273857116699 seconds ---
--- 0.0413973331451416 seconds ---
--- 0.031931400299072266 seconds ---
--- 0.04188966751098633 seconds ---
--- 0.041887760162353516 seconds ---
--- 0.03493475914001465 seconds ---
--- 0.03690600395202637 seconds ---
--- 0.035904645919799805 seconds ---
--- 0.030946731567382812 

--- 0.04484963417053223 seconds ---
--- 37.77992916107178 seconds ---
[['chr6:10002', 'chr14:11909'], ['chr12:30122'], ['chr4:100351'], ['chr1:1']]
